In [1]:
from src import functions as fncs
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [2]:
# read files with data
gdf_areas = gpd.read_file('data/areas_SUM.shp')
gdf_zones = gpd.read_file('data/krk.geojson')
gdf_centroid = gpd.read_file('data/krk_centroid.geojson')

In [3]:
# read ODMs and calculate probabilities
od = pd.read_excel('data/krk_ODM.xlsx', 'AM_peak_PuT')
od.rename(columns = {'464 x 464' : 'zone_NO', 'Unnamed: 2': 'sum'}, inplace = True)
od.drop([0, 1], inplace=True)
od.drop(od.filter(regex="Unname"), axis=1, inplace=True)
od.reset_index(drop=True, inplace=True)
od[0:] = od[:].astype(float)
od['zone_NO'] = od['zone_NO'].astype(int)
# filter OD zone number to match city zones
od_filtered = od.loc[od['zone_NO'].isin(gdf_zones['NO']), ['zone_NO', 'sum'] + gdf_zones['NO'].to_list()]
# calculate probabilities for each origin point
od_probs = od_filtered.loc[:, 'sum':].astype('float64').divide(od_filtered.loc[:, 'sum'].astype('float64'), axis=0)
od_probs.insert(0, 'zone_NO', od_filtered['zone_NO'])

In [4]:
# read demographic of the city
df = pd.read_csv('data/krk_demographic.csv')
df = df.rename(columns={"adr_pelny": "address", "ogolem": "total" })
# assign zone number to each address point
df['zone_NO'] = df.apply(lambda row: fncs.find_containing_polygon(Point(row['x'], row['y']), gdf_zones), axis=1)
# remove the trips with nan zone
df.dropna(subset='zone_NO', inplace=True, ignore_index=True)

In [5]:
fncs.calc_demand(gdf_areas, df, gdf_centroid, od_probs)